# Agent Based Model: Main Notebook

In [358]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Creating Study Groups

#### Practical Functions

In [359]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(2.5, 2.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000)

In [360]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = [] 
    social_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)
        social_list.append(student.Social_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list,
                        'Social': social_list})
    
    return data
    

#### Create Study Group

In [361]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ## 
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic and Social Skills ##
        self.Academic_Skill = 0
        self.Social_Skill = 0

        ## Own Solution ##
        self.Own_Solution = []
        self.Updated_Solution = []

        ## Presenting = True if the student is presenting, False if not ## The default will be false
        self.presenting = False


def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = max_vals[0]
        del max_vals[0]
    else:
        student.ExScore = min_vals[0]
        del min_vals[0]
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = max_vals[0]
        del max_vals[0]
    else:
        student.SeScore = min_vals[0]
        del min_vals[0]
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = max_vals[0]
        del max_vals[0]
    else:
        student.ThScore = min_vals[0]
        del min_vals[0]

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = max_vals[0]
        del max_vals[0]
    else:
        student.JuScore = min_vals[0]
        del min_vals[0]
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.ExScore)*0.15 + (1-student.SeScore)*0.35 + student.ThScore*0.20 + (1-student.JuScore)*0.30
    student.Social_Skill = student.ExScore*0.50 + (1-student.SeScore)*0.10 + student.ThScore*0.30 + (1-student.JuScore)*0.10

In [362]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [363]:
def true_solution_generator(n_elements, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_elements):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [364]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    for student in studygroup[1]: # loop for each individual
        Own_Solution_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, student.Academic_Skill, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Own_Solution_lst.append(true_solution[i])
            else:
                Own_Solution_lst.append(random.randint(range_elements[0], range_elements[1]))
        
        student.Own_Solution = Own_Solution_lst
        student.Updated_Solution = Own_Solution_lst

## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [365]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    presenter_order = []
    proposed_solutions = []

    while (Alfa.Updated_Solution != Bravo.Updated_Solution) and (Bravo.Updated_Solution != Charlie.Updated_Solution) and (Charlie.Updated_Solution != Delta.Updated_Solution) or (n_ticks != max_ticks):
        
        # Starting a Round
        presenter_Name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        presenter_order.append(presenter_Name)

        for student in student_list:
            if student.Name == presenter_Name:
                Proposed_Solution = student.Updated_Solution
                proposed_solutions.append(student.Updated_Solution)
        
        for student in student_list:
                for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                    coin_toss = np.random.binomial(1, student.Social_Skill, 1)[0] #high social skill has a greater chance of accepting the proposal
                    if coin_toss == 1:
                        student.Updated_Solution[i] = Proposed_Solution[i]
                    else:
                        pass
        

        n_ticks = n_ticks + 1 #adding a tick to the simulation
        for student in student_list:
            student.presenting = False

    
    return Proposed_Solution, n_ticks, presenter_order, proposed_solutions

# TEST RUN

In [394]:
student_list_test = ["ESTJ", "ISTJ", "ENTP", "INFJ"]

In [395]:
studygroup1 = StudyGroup(student_list_test)

In [396]:
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic    Social
0     Alfa  ESTJ  0.707482  0.778461  0.747137  0.515146  0.416300  0.648521
1    Bravo  ISTJ  0.204756  0.516634  0.890998  0.550309  0.601571  0.462983
2  Charlie  ENTP  0.605923  0.472525  0.671568  0.266209  0.598179  0.630558
3    Delta  INFJ  0.355368  0.262014  0.191116  0.619965  0.507224  0.346821


In [397]:
true_solution_test = true_solution_generator(10, [1, 9])

In [398]:
true_solution_test[0]

[4, 4, 9, 1, 1, 9, 3, 9, 3, 8]

In [399]:
individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])

In [404]:
print(studygroup1[1][0].Own_Solution)
print(studygroup1[1][1].Own_Solution)
print(studygroup1[1][2].Own_Solution)
print(studygroup1[1][3].Own_Solution)

[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 5, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]


In [403]:
print(studygroup1[1][0].Updated_Solution)
print(studygroup1[1][1].Updated_Solution)
print(studygroup1[1][2].Updated_Solution)
print(studygroup1[1][3].Updated_Solution)

[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 5, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]
[6, 4, 9, 7, 8, 9, 5, 7, 8, 8]


In [402]:
collaborative_solution(studygroup1, 10)

([6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
 10,
 ['Alfa',
  'Charlie',
  'Alfa',
  'Charlie',
  'Delta',
  'Delta',
  'Alfa',
  'Bravo',
  'Alfa',
  'Delta'],
 [[6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 5, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8],
  [6, 4, 9, 7, 8, 9, 5, 7, 8, 8]])